In [1]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.chat_models import ChatOllama
from langchain import PromptTemplate
from langchain_community.vectorstores import FAISS
import json

/Users/karanghai/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

/Users/karanghai/Library/Python/3.9/lib/python/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [229]:
with open('chunks.json', 'r') as file:
    chunks = json.load(file)
chunks

['OK, good morning, everybody. So last week, we stopped at the question, how do we calculate the gradients for more complex loss functions and classifiers? And we saw that the thing that we need is the backpropagation algorithm. So we want to make use of the chain rule of calculus to determine the gradient of some complex compute graph. So we take one compute graph where we know the gradient for each of the nodes, and we use the chain rule to multiply those individual gradients, and thereby getting the gradient for the more complex function. And if, for example, we use a the compute graph for logistic regression, where we say, OK, we have two inputs, so two parameters for each of the input dimensions, and one bias term, our compute graph is this dot product over here, adding the bias term, getting this linear unit over here, taking the sigma. OK. And then we can use the sigma function of all of this and doing, performing, calculating our loss function. And for each of those steps, we k

In [74]:
db2 = FAISS.from_texts(chunks, embeddings)

In [215]:
# DEFAULT_SYSTEM_PROMPT = """
# You are Karan, a student at TH Bingen University. Act as Karan and reply all questions on his behalf. Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. 
# """.strip()


# def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
#     return f"""
#       <|system|>

#       {system_prompt}

#       <|user|>

#       {prompt}

#       <|assistant|>
# """.strip()

In [216]:
DEFAULT_SYSTEM_PROMPT = """
You are Karan, a student at TH Bingen University. Act as Karan and reply all questions on his basis. Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. 
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()

In [217]:
template = generate_prompt(
    """
{context}

Question: {question}
"""
)

In [218]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [220]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOllama(model="llama2:latest"),
    chain_type="stuff",
    retriever=db2.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [223]:
from flask import Flask, request, render_template_string

app = Flask(__name__)

html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Chatbot</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f0f0f0;
        }
        .chat-container {
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
            background-color: #ffffff;
            border-radius: 10px 10px 0px 0px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        .chat-form{
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
            background-color: #ffffff;
            border-radius: 0px 0px 10px 10px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        .chat-message {
            background-color: #f9f9f9;
            border-radius: 10px;
            padding: 10px;
            margin: 10px 0;
        }
        .user-message {
            text-align: right;
        }
        input[type="text"] {
            width: calc(100% - 100px); /* Adjust width of input */
            padding: 8px;
            border-radius: 5px;
            border: 1px solid #ccc;
            margin-right: 10px;
        }
        #input_submit{
            background-color: #555555;
            border: none;
            color: white;
            padding: 8px 10px;
            text-align: center;
            text-decoration: none;
            display: inline-block;
            font-size: 16px;
            margin: 4px 2px;
            cursor: pointer;
            border-radius: 5px;
        }
    </style>
</head>
<body>
    <div class="chat-container" id="chat-container">
    <h3>Welcome to the Artificial Intelligence QnA service.  You may ask me anything 🙃</h3>
        {% for message in chat_history %}
            <div class="chat-message {% if message['sender'] == 'user' %}user-message{% endif %}">
                {{ message['content'] }}
            </div>
        {% endfor %}
    </div>
    <div class="chat-form">
    <form action="/" method="POST" id="chat-form">
        <input type="text" name="input_text" id="input_text" placeholder="Message Chatbot...">
        <input type="submit" id="input_submit" value="Send">
    </form>
    </div>
    <script>
        document.getElementById('chat-form').addEventListener('submit', function(event) {
            event.preventDefault(); // Prevent default form submission
            var inputText = document.getElementById('input_text').value;
            if (inputText.trim() !== '') {
                var inputBox = document.getElementById('input_text');
                console.log("inputBox", inputBox);
                var chatContainer = document.getElementById('chat-container');
                var userMessage = document.createElement('div');
                userMessage.className = 'chat-message user-message';
                userMessage.textContent = inputText;
                chatContainer.appendChild(userMessage);
                document.getElementById('chat-form').submit(); // Submit form
            }
        });
    </script>
    
</body>
</html>
"""

chat_history = []

@app.route('/', methods=['GET', 'POST'])
def index():
    global chat_history

    if request.method == 'POST':
        input_text = request.form['input_text']
        inputAfterSimilaritySearch = db2.similarity_search(query=input_text, k=2)
        output_text = qa_chain({"input_documents": inputAfterSimilaritySearch[0].page_content, "query": input_text})
        chat_history.append({'sender': 'user', 'content': input_text})

        # Add logic here to generate response based on input_text
        # For demonstration, simply echo the input_text as response
        response_text = output_text['result']
        chat_history.append({'sender': 'bot', 'content': response_text})

    return render_template_string(html_template, chat_history=chat_history)

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=81)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:81
 * Running on http://172.20.10.4:81
Press CTRL+C to quit
127.0.0.1 - - [12/Mar/2024 06:31:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2024 06:31:21] "POST / HTTP/1.1" 200 -


In [228]:
# from flask import Flask, request, jsonify

# app = Flask(__name__)

# response_text = ''

# @app.route('/', methods=['GET', 'POST'])
# def index():
#     global response_text

#     if request.method == 'POST':
#         input_text = request.form['input_text']
#         inputAfterSimilaritySearch = db2.similarity_search(query=input_text, k=2)
#         output_text = qa_chain({"input_documents": inputAfterSimilaritySearch[0].page_content, "query": input_text})
#         response_data = {
#             'result': output_text['result'],
#             'source': inputAfterSimilaritySearch[0].page_content
#         }
#         return jsonify(response_data)

# if __name__ == '__main__':
#     app.run(host="0.0.0.0", port=81)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:81
 * Running on http://172.20.10.4:81
Press CTRL+C to quit


127.0.0.1 - - [12/Mar/2024 06:38:28] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2024 06:38:36] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2024 06:38:44] "POST / HTTP/1.1" 200 -


In [ ]:
from flask import Flask, request, render_template_string

app = Flask(__name__)

html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Chatbot</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f0f0f0;
        }
        .chat-container {
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
            background-color: #ffffff;
            border-radius: 10px 10px 0px 0px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        .chat-form{
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
            background-color: #ffffff;
            border-radius: 0px 0px 10px 10px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        .chat-message {
            background-color: #f9f9f9;
            border-radius: 10px;
            padding: 10px;
            margin: 10px 0;
        }
        .user-message {
            text-align: right;
        }
        input[type="text"] {
            width: calc(100% - 100px); /* Adjust width of input */
            padding: 8px;
            border-radius: 5px;
            border: 1px solid #ccc;
            margin-right: 10px;
        }
        #input_submit{
            background-color: #555555;
            border: none;
            color: white;
            padding: 8px 10px;
            text-align: center;
            text-decoration: none;
            display: inline-block;
            font-size: 16px;
            margin: 4px 2px;
            cursor: pointer;
            border-radius: 5px;
        }
    </style>
</head>
<body>
    <div class="chat-container" id="chat-container">
    <h3>Welcome to the Artificial Intelligence QnA service.  You may ask me anything 🙃</h3>
        {% for message in chat_history %}
            <div class="chat-message {% if message['sender'] == 'user' %}user-message{% endif %}">
                {{ message['content'] }}
            </div>
        {% endfor %}
    </div>
    <div class="chat-form">
    <form action="/" method="POST" id="chat-form">
        <input type="text" name="input_text" id="input_text" placeholder="Message Chatbot...">
        <input type="submit" id="input_submit" value="Send">
    </form>
    </div>
    <script>
        document.getElementById('chat-form').addEventListener('submit', function(event) {
            event.preventDefault(); // Prevent default form submission
            var inputText = document.getElementById('input_text').value;
            if (inputText.trim() !== '') {
                var inputBox = document.getElementById('input_text');
                console.log("inputBox", inputBox);
                var chatContainer = document.getElementById('chat-container');
                var userMessage = document.createElement('div');
                userMessage.className = 'chat-message user-message';
                userMessage.textContent = inputText;
                chatContainer.appendChild(userMessage);
                document.getElementById('chat-form').submit(); // Submit form
            }
        });
    </script>
    
</body>
</html>
"""

chat_history = []

@app.route('/', methods=['GET', 'POST'])
def index():
    global chat_history

    if request.method == 'POST':
        input_text = request.form['input_text']
        inputAfterSimilaritySearch = db2.similarity_search(query=input_text, k=2)
        output_text = qa_chain({"input_documents": inputAfterSimilaritySearch[0].page_content, "query": input_text})
        chat_history.append({'sender': 'user', 'content': input_text})

        # Add logic here to generate response based on input_text
        # For demonstration, simply echo the input_text as response
        response_text = output_text['result']
        chat_history.append({'sender': 'bot', 'content': response_text})

    return render_template_string(html_template, chat_history=chat_history)

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=81)